# Описание проекта
- Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
- Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.
- Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.
- Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

# Инструкция по выполнению проекта
1. Загрузите и подготовьте данные. Поясните порядок действий.
2. Исследуйте баланс классов, обучите модель без учёта дисбаланса. Кратко опишите выводы.
3. Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую. Кратко опишите выводы.
4. Проведите финальное тестирование.

# Описание данных
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

    Целевой признак
- Exited — факт ухода клиента

# 1. Подготовка данных

In [1]:
!pip install progress

In [2]:
# Импортируем самую важную библиотеку
import pandas as pd

In [3]:
# Иvпортируем метрику
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [4]:
# Импортируем модели
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 

In [5]:
# Импортируем остальное 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [6]:
# Откроем таблицу двумя путями
server_path = '/datasets/Churn.csv'
local_path = '/Users/mmaximmaximovgmail.com/Desktop/DS/project_7/Churn.csv'

try:
    bank = pd.read_csv(server_path)
except:
    bank = pd.read_csv(local_path)

In [7]:
# Выводим первые 5 строк таблицы
bank.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [8]:
# Кол-во строк и колонок
bank.shape

(10000, 14)

In [9]:
# Среднее, стандартное и минимальное значение 
bank.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [10]:
# Типы и кол-во пропусков
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [11]:
# Кол-во дубликатов
bank.duplicated().sum()

0

In [12]:
# Убираем не нужные колонки в таблице, они не пригодятся для обучения модели
# inplace=True помогает сразу внести изменения в ту же самую перменную 
bank.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
print(bank.shape, 'Удалили не нужные столбцы')

(10000, 11) Удалили не нужные столбцы


In [13]:
# Теперь должны заполнить пропуски в столбце Tenure, в течение проработки модели может поменяем значение
bank['Tenure'] = bank['Tenure'].fillna(-1)

In [14]:
# Заменю тип колонки Tenure
#bank['Tenure'] = bank['Tenure'].astype('object')

In [15]:
# Преобразуем колонки в таблице техникой OHE, чтобы не попасть в дамми ловушку используем drop_first
bank = pd.get_dummies(bank, drop_first=True)

### 1.2 Разбиваем данные на выборки: обучающу, валидационную, тестовую


In [16]:
# Обучающая выборка
feature = bank.drop('Exited', axis=1)
target = bank['Exited']

In [17]:
# Возьмем 60% для обучающей выборки
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.4, random_state=12345, )


In [18]:
# Делим оставшиеся 40%, по полам между тестовой и валидационной
feature_test, feature_valid, target_test, target_valid = train_test_split(feature_test, target_test, test_size=0.5, random_state=12345)


In [19]:
# Выведим для проверки, что все разделилось в соотношение 3:1:1
display(feature_train.shape, target_train.shape)
display(feature_test.shape, target_test.shape)
display(feature_valid.shape, target_valid.shape)

(6000, 11)

(6000,)

(2000, 11)

(2000,)

(2000, 11)

(2000,)

### 1.3 Масштабируем данные в некоторых колонках, чтобы избежать ловушки с большим разбросом значений, поэтому приведем все к одному признаку

In [20]:
# Для масштабирывания я выбрал 6 колонок, так как они имеют большие значения и для того, чтоб не было проблем,
# я сейчас обработаю их
# P.S Делаю по примеру из задания `Масштабирование признаков`

numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(feature_train[numeric])
feature_train[numeric] = scaler.transform(feature_train[numeric])
feature_valid[numeric] = scaler.transform(feature_valid[numeric])
feature_test[numeric] = scaler.transform(feature_test[numeric])

display(feature_train.sample(2), 'Обучающая')
display(feature_valid.sample(2), 'Валидационная')
display(feature_test.sample(2), 'Тестовая')

/Users/mmaximmaximovgmail.com/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
8517,0.369812,-1.796742,1.083651,0.982515,0.830152,1,1,1.698754,0,0,1
569,1.127903,-0.752805,0.776349,-1.233163,2.551864,1,1,1.163860,0,0,1


'Обучающая'

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7858,-0.928290,-1.606935,-0.452862,0.981849,-0.891560,0,1,-1.622890,0,0,1
6189,0.639817,0.006422,-0.452862,1.179944,0.830152,1,0,-0.617439,1,0,1


'Валидационная'

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
3223,1.283676,0.006422,-0.760164,0.446806,-0.89156,0,0,-0.346279,0,1,1
8303,-1.364452,0.386035,-1.682072,-1.233163,-0.89156,1,1,-0.109405,0,0,1


'Тестовая'

# 2 Исследование задачи

In [21]:
# Начнем с модели дерева
best_model = None
best_result = 0
for depth in tqdm(range(1, 14)):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(feature_train, target_train)
    prediction = model.predict(feature_valid)
    result = accuracy_score(target_valid, prediction)
    print('Глубина дереа:', depth, 'Точность:', result)
    if result > best_result:
        best_model=depth
        best_result=result
        
print('Подходящая глубина :', best_model, 'Подходящая точность:', best_result)
print('f1-мера :', f1_score(target_valid, prediction))
print(confusion_matrix(target_valid, prediction))

  0%|                                                    | 0/13 [00:00<?, ?it/s]

Глубина дереа: 1 Точность: 0.7885
Глубина дереа: 2 Точность: 0.817
Глубина дереа: 3 Точность: 0.83
Глубина дереа: 4 Точность: 0.832
Глубина дереа: 5 Точность: 0.842
Глубина дереа: 6 Точность: 0.848
Глубина дереа: 7 Точность: 0.8365
Глубина дереа: 8 Точность: 0.835


 69%|██████████████████████████████▍             | 9/13 [00:00<00:00, 87.82it/s]

Глубина дереа: 9 Точность: 0.828


100%|███████████████████████████████████████████| 13/13 [00:00<00:00, 71.85it/s]

Глубина дереа: 10 Точность: 0.832
Глубина дереа: 11 Точность: 0.8125
Глубина дереа: 12 Точность: 0.8095
Глубина дереа: 13 Точность: 0.81
Подходящая глубина : 6 Подходящая точность: 0.848
f1-мера : 0.503916449086162
[[1427  150]
 [ 230  193]]


In [22]:
# Модель случайного леса
import itertools
import numpy as np

best_params = None
best_result = 0

for params in tqdm(itertools.product(range(1, 16), range(1, 21), range(1, 5))):
    depth, est, min_s_l = params
    model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est, min_samples_leaf=min_s_l)
    model.fit(feature_train, target_train)
    result = model.score(feature_valid, target_valid)
    prediction = model.predict(feature_valid)
    if result > best_result:
        best_params = params
        best_result = result
        
print(f'Score of best model {np.round(best_result, 5)}')
print(f'\nParams:\nmax_depth = {best_params[0]}\nn_estimators = {best_params[1]}\nmin_samples_leaf = {best_params[2]}')
print('f1-мера :', f1_score(target_valid, prediction))
print(confusion_matrix(target_valid, prediction))

1200it [00:48, 24.52it/s]

Score of best model 0.8515

Params:
max_depth = 14
n_estimators = 20
min_samples_leaf = 1
f1-мера : 0.5246913580246915
[[1522   55]
 [ 253  170]]


In [23]:
# Модель логистической регрессии
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(feature_train, target_train)
result = model.score(feature_valid, target_valid)
prediction = model.predict(feature_valid)
print('Accuracy:', result)
print('f1-мера :', f1_score(target_valid, prediction))
print(confusion_matrix(target_valid, prediction))

Accuracy: 0.7915
f1-мера : 0.27478260869565213
[[1504   73]
 [ 344   79]]


### Мини вывод 2 шагу
- Низкий показатель f1 показывает о низком качестве модели
- Матрица неточностей показывает высокое FP показание, что не очень хорошо(модель думает об одном а ответ другой)

# 3 Борьба с дисбалансом

### 3.1 Начнем с Upsample + перемешаем данные

In [24]:
def upsample(feature, target, repeat):
    feature_zeros = feature[target == 0]
    feature_ones = feature[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    feature_upsampled = pd.concat(
        [feature_zeros]+[feature_ones] * repeat)
    target_upsampled = pd.concat(
        [target_zeros]+[target_ones] * repeat)
    feature_upsampled, target_upsampled = shuffle(feature_upsampled, target_upsampled, random_state=12345)
    return feature_upsampled, target_upsampled
feature_upsampled, target_upsampled = upsample(feature_train, target_train, 4)
print(feature_upsampled.shape)
print(target_upsampled.shape)

model = RandomForestClassifier(random_state=12345, max_depth=14)
model.fit(feature_upsampled, target_upsampled)
predicted_valid = model.predict(feature_valid)
print('F1 :', f1_score(target_valid, predicted_valid))

(9588, 11)
(9588,)
F1 : 0.6019900497512438


### 3.2 Закончим с downsample + пермешаем

In [25]:
def downsample(feature, target, fraction):
    feature_zeros = feature[target == 0]
    feature_ones = feature[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    feature_downsampled = pd.concat(
        [feature_zeros.sample(frac=fraction, random_state=12345)] + [feature_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    feature_downsampled, target_downsampled = shuffle(
        feature_downsampled, target_downsampled, random_state=12345)
    
    return feature_downsampled, target_downsampled
feature_downsampled, target_downsampled = downsample(feature_train, target_train, 0.5)
print(feature_downsampled.shape)
print(target_downsampled.shape)

model = RandomForestClassifier(random_state=12345, max_depth=14)
model.fit(feature_downsampled, target_downsampled)
predicted_valid = model.predict(feature_valid)
print('F1 :', f1_score(target_valid, predicted_valid))

(3598, 11)
(3598,)
F1 : 0.6051660516605165


# 4 Тестирование модели

In [26]:
test_predictions = model.predict(feature_test)
print('Тестовая выборка:', accuracy_score(target_test, test_predictions))
print('F1 :', f1_score(target_test, test_predictions))

Тестовая выборка: 0.8415
F1 : 0.6138855054811206


# 5 Итоговый вывод
- Описал и оформил описание проекта, план действий, описание данных в таблице и начал выполнять поэтапно план
- Сначала испортировал библиотеки для анализа данных и работы с ними
- Посмотрел таблице, тип, среднее/медианное/минимальное значение, кол-во дубликатов

        Дальше пошло интересней
- Убрали 3 не нужных колонок в таблице, так как они не несут никакой информации для анализа
- Добавили в колонку `Tenure` на все пустые значения -1, чтобы не помешало при анализе
- Преобразовал колонки техникой `OHE` и `drop_first`
- Разбили данные выборки в соотношение 3:1:1 (обущающая, валидационная, тестова), перепроверили, что верно разбили
- Масштобировали 6 колонок из таблицы ( более подробно описал в шаге 1.3)
- Исследовали 3 модели: Дерево, случайный лес, логистическая регрессия
- Высокий показатель F1 в модели показал случайный лес со значением 0.525, худшим логистическая регрессия 
- НО матреца неточностей показало высокое показание FP, что не очень хорошо
- Побороли дисбаланас с помощью `upsample`  `downsample` и перемешиванием 
- Благодаря случайному лесу в дисбалансе, F1-мера показало отличный результат: `0.602`  `0.605`
- И в тестировании модели по лучшиму показателю я взял тот же самый случайный лес